In [3]:
import os.path as op
import os
import numpy as np
from ipyparallel import Client

In [4]:
engines = Client()

SSHException: The authenticity of the host can't be established.

In [ ]:
home_dir =  '/home/dnl/habitization/'
data_dir = home_dir + '/data/'
subs = home_dir + 'subjects.txt'
subs = list(np.loadtxt(subs,str))

ignore_subs = ['HAB02', 'HAB03', 'HAB05', 'HAB08', 'HAB10', 'HAB11', 'HAB12', 'HAB13', 'HAB14', 'HAB16', 'HAB17']
#First analysis included subjects 'HAB02', 'HAB03', 'HAB05', 'HAB08', 'HAB10', 'HAB11', 'HAB12', 'HAB13', 'HAB14', 'HAB16'
#Second analysis included subjects 'HAB01', 'HAB07', 'HAB18', 'HAB19', 'HAB20', 'HAB21', 'HAB22', 'HAB23', 'HAB24', 'HAB25'
subs = [x for x in subs if x not in ignore_subs]

In [ ]:
def bet(sub):
    T1s = ['T1_a','T1_b']
    for t1 in T1s:
        anat = op.join(data_dir,sub,'anat',t1 + '.nii.gz')
        new_anat = op.join(data_dir,sub,'anat',t1 + '_brain.nii.gz')
        
        if not op.exists(new_anat):
            cmd = ['bet',anat,new_anat,'-R','-f','.4']
            cmd = ' '.join(cmd)        
            os.system(cmd)
    return sub

In [ ]:
sub_engines = engines[:] #less if you want to use fewer
sub_engines.push(dict(bet = bet,
                     data_dir = data_dir))
sub_engines.execute('import numpy as np')
sub_engines.execute('import os.path as op')
with sub_engines.sync_imports():
    import os
    
output = sub_engines.map_sync(bet,subs)